In [3]:
import pandas as pd
import numpy as np
import joblib

from keras.models import load_model

from art.attacks.evasion import DeepFool, CarliniL2Method
from art.attacks.evasion.fast_gradient import FastGradientMethod
from art.estimators.classification import KerasClassifier

from adversarialdefence.utils import ModelUtils, GeneralUtils

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/lib/python3.11/site-packages/art/estimators/certification/__init__.py:29: UserWarning: PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality
  warnings.warn("PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality")


### ***Data Loading***

In [4]:
df = pd.read_csv('../csv/CICIDS2017_improved-preprocessed.csv')
df.drop(df.columns[0], axis=1, inplace=True)
df.shape

(1715326, 49)

In [5]:
df_benign = df[df['Label'] == 0]
df_anomalous = df[df['Label'] == 1]

print(f'Number of benign samples: {df_benign.shape[0]}')
print(f'Number of anomalous samples: {df_anomalous.shape[0]}')

Number of benign samples: 1432918
Number of anomalous samples: 282408


### ***Load models and scalers***

In [6]:
autoencoder = load_model('../modelli/autoencoder_best_weights_96-96.hdf5')
dnn = load_model('../modelli/DNN_best_weights_99.hdf5')

In [7]:
std_scaler_aut = joblib.load('../modelli/std_scaler_aut.bin')
std_scaler_dnn = joblib.load('../modelli/std_scaler_dnn.bin') 

/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
def checkAdversarialResults(X_advs, df_advs, attack_type, num_total_attempts, dnn_target, aut_binary_th=0.05, context=''):
    preds_advs_dnn = ModelUtils.binary_preds_supervised(dnn_target, X_advs)
    advs_samples_dnn = GeneralUtils.get_advs_samples(np.array(preds_advs_dnn), df_advs)
    print(f'Successful adversarial samples of type {attack_type} for DNN {context}: {advs_samples_dnn.shape[0]} / {num_total_attempts} - ASR: {(advs_samples_dnn.shape[0]/num_total_attempts) * 100}%')

    preds_advs_aut = ModelUtils.binary_preds_unsupervised(autoencoder, X_advs, aut_binary_th)
    advs_samples_aut = GeneralUtils.get_advs_samples(np.array(preds_advs_aut), df_advs)
    print(f'Successful adversarial samples of type {attack_type} for AUT {context}: {advs_samples_aut.shape[0]} / {num_total_attempts} - ASR: {(advs_samples_aut.shape[0]/num_total_attempts) * 100}%')

    preds_advs_dnn_by_aut = ModelUtils.binary_preds_unsupervised(autoencoder, advs_samples_dnn.copy().drop('Label', axis=1), aut_binary_th)
    common_advs_samples = GeneralUtils.get_advs_samples(np.array(preds_advs_dnn_by_aut), advs_samples_dnn)
    print(f'Successful adversarial samples of type {attack_type} for TARGET SYSTEM {context}: {common_advs_samples.shape[0]} / {num_total_attempts} - ATSR: {(common_advs_samples.shape[0]/num_total_attempts) * 100}%')

### ***Perform WB Attacks***

In [9]:
BASE_ADVS_CSV_PATH = '../csv/'
X_anomalous_to_perturb = std_scaler_dnn.transform(df_anomalous.drop('Label', axis=1))

In [10]:
#tf.compat.v1.disable_eager_execution()
#dnn = load_model('../modelli/DNN_best_weights_99.hdf5')
#dnn_target = KerasClassifier(model=dnn, clip_values=(X_anomalous_to_perturb.min(), X_anomalous_to_perturb.max()))

**FGSM**

In [9]:
#fgsm_attack = FastGradientMethod(dnn_target, eps=0.07)
#fgsm_advs = fgsm_attack.generate(X_anomalous_to_perturb)

In [11]:
df_fgsm, X_fgsm, y_fgsm = GeneralUtils.get_data_with_advs(BASE_ADVS_CSV_PATH + 'fgsm.csv')
checkAdversarialResults(X_fgsm, df_fgsm, 'FGSM', X_anomalous_to_perturb.shape[0], dnn)

Successful adversarial samples of type FGSM for DNN : 94759 / 282408 - ASR: 33.55393614911759%
Successful adversarial samples of type FGSM for AUT : 16858 / 282408 - ASR: 5.969377638027251%
Successful adversarial samples of type FGSM for TARGET SYSTEM : 4645 / 282408 - ATSR: 1.6447834338970568%


**Deepfool Attack**

In [11]:
#deepFool_attack = DeepFool(dnn_target, max_iter=1)
#deepFool_advs = deepFool_attack.generate(X_anomalous_to_perturb, eps=1e-6)

In [12]:
df_deepFool, X_deepFool, y_deepFool = GeneralUtils.get_data_with_advs(BASE_ADVS_CSV_PATH + 'deepFool.csv')
checkAdversarialResults(X_deepFool, df_deepFool, 'DEEPFOOL', X_anomalous_to_perturb.shape[0], dnn)

Successful adversarial samples of type DEEPFOOL for DNN : 238261 / 282408 - ASR: 84.36765247443415%
Successful adversarial samples of type DEEPFOOL for AUT : 24430 / 282408 - ASR: 8.650604798730914%
Successful adversarial samples of type DEEPFOOL for TARGET SYSTEM : 206 / 282408 - ATSR: 0.0729441092320331%


**Carlini Attack**

In [13]:
#carlini_attack = CarliniL2Method(dnn_target, max_iter=2, batch_size=1, targeted=False)
#step = 5043

#for i in range(53, 57):
#    start = step * (i - 1)
#    end = step * i
#    X_anomalous_to_perturb_process = X_anomalous_to_perturb[start:end]
#    carlini_advs = carlini_attack.generate(X_anomalous_to_perturb_process)
#    df_carlini = pd.DataFrame(carlini_advs, columns=columns).to_csv('carlini_' + str(i) + '.csv')

In [13]:
df_carlini, X_carlini, y_carlini= GeneralUtils.get_data_with_advs(BASE_ADVS_CSV_PATH + 'carlini.csv')
X_carlini.drop(X_carlini.columns[0], axis=1, inplace=True)
df_carlini.drop(df_carlini.columns[0], axis=1, inplace=True)

checkAdversarialResults(X_carlini, df_carlini, 'CARLINI ', X_anomalous_to_perturb.shape[0], dnn)

Successful adversarial samples of type CARLINI  for DNN : 25331 / 282408 - ASR: 8.96964675221665%
Successful adversarial samples of type CARLINI  for AUT : 229525 / 282408 - ASR: 81.27425568680773%
Successful adversarial samples of type CARLINI  for TARGET SYSTEM : 11243 / 282408 - ATSR: 3.9811195150279026%
